In [2]:
!pip install faker crewai transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.5 MB/s eta 0:0

In [1]:
pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 57.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [2]:
from faker import Faker

faker = Faker()

FAKER_FUNCTIONS = {}
for provider in faker.providers:
    for func in dir(provider):
        if callable(getattr(provider, func, None)) and not func.startswith("_"):
            FAKER_FUNCTIONS[func] = getattr(faker, func, None)

print("\n".join(FAKER_FUNCTIONS.keys()))


android_platform_token
bothify
chrome
firefox
hexify
internet_explorer
ios_platform_token
language_code
lexify
linux_platform_token
linux_processor
locale
mac_platform_token
mac_processor
numerify
opera
random_choices
random_digit
random_digit_above_two
random_digit_not_null
random_digit_not_null_or_empty
random_digit_or_empty
random_element
random_elements
random_int
random_letter
random_letters
random_lowercase_letter
random_number
random_sample
random_uppercase_letter
randomize_nb_elements
safari
user_agent
windows_platform_token
ein
invalid_ssn
itin
ssn
sbn9
enum
pybool
pydecimal
pydict
pyfloat
pyint
pyiterable
pylist
pyobject
pyset
pystr
pystr_format
pystruct
pytuple
profile
simple_profile
basic_phone_number
country_calling_code
msisdn
phone_number
first_name
first_name_female
first_name_male
first_name_nonbinary
language_name
last_name
last_name_female
last_name_male
last_name_nonbinary
name
name_female
name_male
name_nonbinary
prefix
prefix_female
prefix_male
prefix_nonbinary
su

In [27]:
import re
import string
import numpy as np
from transformers import pipeline
from faker import Faker
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util

nlp = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
faker = Faker()
model = SentenceTransformer("all-MiniLM-L6-v2")

# List available Faker functions
FAKER_FUNCTIONS = {}
for provider in faker.providers:
    for func in dir(provider):
        if callable(getattr(provider, func, None)) and not func.startswith("_"):
            FAKER_FUNCTIONS[func] = getattr(faker, func, None)

# Precompute embeddings for Faker function names
function_names = list(FAKER_FUNCTIONS.keys())
function_embeddings = model.encode(function_names, convert_to_tensor=True)

STOPWORDS = set(stopwords.words("english"))

def preprocess_query(query):

    query = query.lower()
    query = query.translate(str.maketrans("", "", string.punctuation))
    words = query.split()
    words = [word for word in words if word not in STOPWORDS]
    return " ".join(words)

def extract_age_range(query):
    match = re.findall(r'\d+', query)
    if len(match) == 2:
        return int(match[0]), int(match[1])
    return None, None

def identify_faker_function(query):

    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, function_embeddings)[0]
    best_match_idx = similarities.argmax().item()
    return function_names[best_match_idx]

def generate_faker_output(user_query):
    cleaned_query = preprocess_query(user_query)
    function_name = identify_faker_function(cleaned_query)

    if "date of birth" in user_query and "between" in user_query:
        min_age, max_age = extract_age_range(user_query)
        if min_age and max_age:
            return f"faker.date_of_birth(minimum_age={min_age}, maximum_age={max_age})"

    if all(word in user_query for word in ["name", "date of birth", "email"]):
      return "faker.name(), faker.date_of_birth(), faker.email()"

    if "email" in user_query:
       match = re.search(r'\d+', user_query)
       if match:
          count = int(match.group())
          return f"[faker.{function_name}() for _ in range({count})]"
       return f"faker.{function_name}()"


    if "date of birth" in user_query and "under"or"below" in user_query and "over"or"above" in user_query:
      return "Contradictory request. Please specify either 'under 18' or 'over 30'."


    if function_name in FAKER_FUNCTIONS:
        return f"faker.{function_name}()"

    return " Did you mean: " + ", ".join(function_names[:3]) + "? Please specify for details."

while True:
    user_input = input("\n🔹 Enter your query (or type 'exit' to stop): ")
    if user_input.lower() == "exit":
        break

    result = generate_faker_output(user_input)
    print(f"\n Suggested Faker Function: {result}")


Device set to use cpu



🔹 Enter your query (or type 'exit' to stop): generate 20 company emails

 Suggested Faker Function: [faker.company_email() for _ in range(20)]

🔹 Enter your query (or type 'exit' to stop): generate fake address

 Suggested Faker Function: faker.address()

🔹 Enter your query (or type 'exit' to stop): exit


In [24]:
[faker.email() for _ in range(20)]

['shawmatthew@example.org',
 'rsanchez@example.net',
 'lindabutler@example.com',
 'williamstravis@example.net',
 'stephen24@example.org',
 'brianna74@example.com',
 'operez@example.net',
 'stephen82@example.org',
 'holtdavid@example.com',
 'perrytracy@example.net',
 'jacksondesiree@example.net',
 'ewoods@example.com',
 'patriciastanley@example.com',
 'yarnold@example.org',
 'robinsondaniel@example.org',
 'khunter@example.org',
 'btrujillo@example.com',
 'wileyangela@example.org',
 'ubaker@example.org',
 'erika65@example.org']

In [53]:
faker.basic_phone_number()

'237-643-8887'

In [54]:
faker.address()

'218 Edward Pike\nEast Elizabeth, IL 36416'

Add the openai premium key below, as free tier key will return an error as limit exceeded..As premium key is not available, testing for this approach is not done.

In [29]:
import os
from litellm import completion

os.environ["OPENAI_API_KEY"] = "Please add the Open API key"
llm = "gpt-4o-mini"


In [30]:
from crewai import Agent, Task, Crew, Process
from faker import Faker

faker = Faker()

# Agent 1: Query Understanding Agent
query_agent = Agent(
    role='Query Understanding Specialist',
    goal='Understand user queries and extract key details like type, constraints, and conditions.',
    backstory="Expert in NLP and understanding ambiguous human language, capable of breaking down queries into structured formats.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

# Agent 2: Reasoning & Validation Agent
reasoning_agent = Agent(
    role='Logical Inference & Validation Specialist',
    goal='Analyze extracted query details, resolve ambiguities, contradictions, and refine for execution.',
    backstory="An AI designed to think critically, identify conflicts, and ensure the query is valid before execution.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

# Agent 3: Execution Agent
execution_agent = Agent(
    role='Faker Function Execution Specialist',
    goal='Map validated query details to the correct Faker function and execute.',
    backstory="An expert in generating synthetic data, capable of dynamically selecting the right Faker function.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)


task1 = Task(
    description="Analyze the user query and extract key components such as type of data requested, filters, and constraints.",
    agent=query_agent,
    expected_output="A structured dictionary containing extracted data type, filters, constraints, and any ambiguities."
)


# Task 2: Reasoning & Validation Task
task2 = Task(
    description="Evaluate the extracted query details, resolve ambiguities, and refine the request for execution.",
    agent=reasoning_agent,
    depends_on=[task1],
    expected_output="A list of the most relevant Faker function(s) that match the user query."

)

# Task 3: Execution Task
task3 = Task(
    description="Use Faker to generate data based on refined query details and return the result.",
    agent=execution_agent,
    depends_on=[task2],
    expected_output="A properly formatted string containing the best Faker function(s) matching the user query."

)

# Crew Definition
crew = Crew(
    agents=[query_agent, reasoning_agent, execution_agent],
    tasks=[task1, task2, task3],
    process=Process.sequential
)

def generate_fake_data(user_query):
    print(f"\n User Query: {user_query}")
    result = crew.kickoff(inputs={task1: user_query})
    print(f"\n Suggested Faker Function: {result}")

while True:
    user_input = input("\n Enter your query (or type 'exit' to stop): ")
    if user_input.lower() == "exit":
        break
    generate_fake_data(user_input)


🔹 Enter your query (or type 'exit' to stop): give me date of birth

🔹 User Query: give me date of birth
# Agent: Query Understanding Specialist
## Task: Analyze the user query and extract key components such as type of data requested, filters, and constraints.


ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}




LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

 Error during LLM call: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


RateLimitError: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}